In [5]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

import os
from tqdm import tqdm

In [6]:
LANGUAGE = "english"
SENTENCES_COUNT = 10
TEXTS_COUNT = 100

In [7]:
test_path_txt = '../SCOTU_data_txt_save/text'
test_path_sum = '../SCOTU_data_txt_save/summary'

In [8]:
summary_gen = []

for file_name in tqdm(os.listdir(test_path_txt)[:TEXTS_COUNT]):
    parser = PlaintextParser.from_file(os.path.join(test_path_txt, file_name), Tokenizer(LANGUAGE))

    summarizer = LuhnSummarizer()
    summarizer.stop_words = get_stop_words(LANGUAGE)

    sentence_txt = ''

    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        sentence_txt = sentence_txt + str(sentence) + "\n"
    summary_gen.append(sentence_txt)


  3%|▎         | 3/100 [00:00<00:04, 19.67it/s]

100%|██████████| 100/100 [00:24<00:00,  4.01it/s]


In [9]:
summary_ref = []

for file_name in tqdm(os.listdir(test_path_sum)[:TEXTS_COUNT]):
    with open(os.path.join(test_path_sum, file_name), 'r') as f:
        text = f.read()
        summary_ref.append(text)

100%|██████████| 100/100 [00:00<00:00, 11108.09it/s]


In [10]:
print(len(summary_gen), len(summary_ref))
if len(summary_ref) != len(summary_gen):
    raise ValueError("Les listes summary_ref et summary_gen doivent avoir la même longueur.")

100 100


In [11]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = []
for i in range(len(summary_gen)):
    score = scorer.score(summary_ref[i], summary_gen[i])
    scores.append(score)
    # print(f"Scores pour le résumé {i+1} :", score)

# Moyennes des scores
avg_scores = {
    'rouge1': sum(s['rouge1'].fmeasure for s in scores) / len(scores),
    'rouge2': sum(s['rouge2'].fmeasure for s in scores) / len(scores),
    'rougeL': sum(s['rougeL'].fmeasure for s in scores) / len(scores),
}

print("Scores ROUGE :", avg_scores)

Scores ROUGE : {'rouge1': 0.4091338647028827, 'rouge2': 0.15522700021606992, 'rougeL': 0.19931883097230327}
